In [1]:
import elevenlabs
import requests
import os
import json
import base64
import pygame
import time

pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


c:\Users\rawat\anaconda3\envs\shorts\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [2]:
text=input('Enter your story')

In [3]:
from unrealspeech import UnrealSpeechAPI

speech_api = UnrealSpeechAPI(api_key='G0d0Orw7FJEvnFxIWtWEb4PjhgtzBwtaOFQJ9PJeynQnAogkbMj3YR')


response = speech_api.speech(
    text=text,
    voice_id="Will",      # Choose voice: Scarlett, Dan, Liv, etc.
    timestamp_type="word",    # word or sentence
    bitrate="192k",
    pitch=0.92
)




In [13]:
response

{'CreationTime': '2025-08-15T08:02:19.47Z',
 'OutputUri': 'https://unreal-expire-in-90-days.s3-us-west-2.amazonaws.com/340dcc77-75c2-4b71-b785-326357b6e455-0.mp3',
 'RequestCharacters': 1553,
 'TaskId': '340dcc77-75c2-4b71-b785-326357b6e455',
 'TaskStatus': 'completed',
 'TimestampsUri': 'https://unreal-expire-in-90-days.s3-us-west-2.amazonaws.com/340dcc77-75c2-4b71-b785-326357b6e455-0.json',
 'VoiceId': 'Will'}

In [4]:
sound=requests.get(response['OutputUri'])
timestampurl=requests.get(response['TimestampsUri'])

In [5]:
with open('timestamps.json','wb') as file:
    file.write(timestampurl.content)


with open('test.mp3','wb') as file:
    file.write(sound.content)

In [6]:
data_str = (timestampurl.content).decode("utf-8")  # bytes → string
data_json = json.loads(data_str) 

In [7]:
data_json

[{'start': 0.5, 'end': 0.6586696915328503, 'word': 'i', 'text_offset': 0},
 {'start': 0.6586696915328503,
  'end': 0.9063492094477018,
  'word': 'love',
  'text_offset': 2},
 {'start': 0.9063492094477018,
  'end': 1.3397883698344233,
  'word': 'juices',
  'text_offset': 7}]

In [22]:
data_json=[{'word': 'kya', 'start': 50.5, 'end': 213.0},
 {'word': 'aapko', 'start': 250.5, 'end': 700.5},
 {'word': 'pta', 'start': 700.5, 'end': 1075.5},
 {'word': 'hai', 'start': 1075.5, 'end': 1300.5},
 {'word': 'bhosademon', 'start': 1313.0, 'end': 2100.5},
 {'word': 'ka', 'start': 2125.5, 'end': 2288.0},
 {'word': 'poora', 'start': 2288.0, 'end': 2638.0},
 {'word': 'naam', 'start': 2638.0, 'end': 2925.5},
 {'word': 'mulla', 'start': 2925.5, 'end': 3288.0},
 {'word': 'sherdil', 'start': 3288.0, 'end': 3738.0},
 {'word': 'hai', 'start': 3750.5, 'end': 4113.0}]

In [10]:
data_json[0]

{'word': 'kya', 'start': 50.5, 'end': 213.0}

In [23]:
for i in data_json:

    
    i['start']=i['start']/1000
    i['end']=i['end']/1000


In [24]:
data_json

[{'word': 'kya', 'start': 0.0505, 'end': 0.213},
 {'word': 'aapko', 'start': 0.2505, 'end': 0.7005},
 {'word': 'pta', 'start': 0.7005, 'end': 1.0755},
 {'word': 'hai', 'start': 1.0755, 'end': 1.3005},
 {'word': 'bhosademon', 'start': 1.313, 'end': 2.1005},
 {'word': 'ka', 'start': 2.1255, 'end': 2.288},
 {'word': 'poora', 'start': 2.288, 'end': 2.638},
 {'word': 'naam', 'start': 2.638, 'end': 2.9255},
 {'word': 'mulla', 'start': 2.9255, 'end': 3.288},
 {'word': 'sherdil', 'start': 3.288, 'end': 3.738},
 {'word': 'hai', 'start': 3.7505, 'end': 4.113}]

In [25]:
import ffmpeg

def group_words_into_phrases(words, max_words_per_phrase=3):
    """Groups individual words (dict format) into phrases based on punctuation or max word count."""
    phrases = []
    current_phrase = []
    if not words:
        return []

    phrase_start_time = words[0]["start"]

    for i, w in enumerate(words):
        word = w["word"]
        start = w["start"]
        end = w["end"]

        current_phrase.append(word)
        should_end_phrase = (
            any(c in word for c in ",.?!")
            or len(current_phrase) >= max_words_per_phrase
            or i == len(words) - 1
        )

        if should_end_phrase:
            phrase_text = " ".join(current_phrase)
            phrases.append({
                "phrase": phrase_text,
                "start": phrase_start_time,
                "end": end
            })
            current_phrase = []
            if i < len(words) - 1:
                phrase_start_time = words[i + 1]["start"]

    return phrases



def add_timed_text(stream, text, start, end, max_font_ratio=0.08, min_font_ratio=0.05,
                   fontcolor="white", outline=6):
    """
    Adds centered text that auto-shrinks for long words.
    """
    font_ratio = max(min_font_ratio, max_font_ratio - (len(text) / 50) * 0.01)

    return stream.drawtext(
        text=text,
        fontfile=r"C:\Windows\Fonts\impact.ttf",
        fontsize=f"w*{font_ratio}",
        fontcolor=fontcolor,
        x="(w-text_w)/2",
        y="(h-text_h)/2",
        borderw=outline,
        bordercolor="black",
        box=0,
        enable=f"between(t,{start},{end})"
    )


def overlay(stream, phrases):
    """
    Apply timed phrases to the video stream.
    phrases: list of dicts with keys 'phrase', 'start', 'end'
    """
    for p in phrases:
        stream = add_timed_text(stream, p["phrase"], p["start"], p["end"])
    return stream


In [26]:
data_json

[{'word': 'kya', 'start': 0.0505, 'end': 0.213},
 {'word': 'aapko', 'start': 0.2505, 'end': 0.7005},
 {'word': 'pta', 'start': 0.7005, 'end': 1.0755},
 {'word': 'hai', 'start': 1.0755, 'end': 1.3005},
 {'word': 'bhosademon', 'start': 1.313, 'end': 2.1005},
 {'word': 'ka', 'start': 2.1255, 'end': 2.288},
 {'word': 'poora', 'start': 2.288, 'end': 2.638},
 {'word': 'naam', 'start': 2.638, 'end': 2.9255},
 {'word': 'mulla', 'start': 2.9255, 'end': 3.288},
 {'word': 'sherdil', 'start': 3.288, 'end': 3.738},
 {'word': 'hai', 'start': 3.7505, 'end': 4.113}]

In [27]:
phrases=group_words_into_phrases(data_json)

In [28]:
phrases

[{'phrase': 'kya aapko pta', 'start': 0.0505, 'end': 1.0755},
 {'phrase': 'hai bhosademon ka', 'start': 1.0755, 'end': 2.288},
 {'phrase': 'poora naam mulla', 'start': 2.288, 'end': 3.288},
 {'phrase': 'sherdil hai', 'start': 3.288, 'end': 4.113}]

In [29]:
data_json[len(data_json)-1]['end']

4.113

In [30]:
input_file = "17169505-hd_1080_1920_30fps.mp4"
audio_file = "output.mp3"
output_file = "asscream.mp4"

audio_duration=data_json[len(data_json)-1]['end']+0.5


video = ffmpeg.input(input_file,stream_loop=-1,t=audio_duration).video
audio = ffmpeg.input(audio_file).audio

video = overlay(video, phrases)

(
    ffmpeg
    .output(video, audio, output_file, vcodec='libx264', acodec='aac', strict='experimental')
    .overwrite_output()
    .run()
)


(None, None)